In [6]:
import numpy as np
import tifffile as tiff
from tkinter import Tk, filedialog
from PIL import Image
import os
import reset
from pathlib import Path

def get_median_array(image):
    """Calculate the median values along the y-direction for each row."""
    h, w = image.shape
    median_array = np.median(image, axis=1)
    return median_array

def get_normalizing_values(median_array):
    """Calculate normalizing coefficients to scale each median value to the maximum median."""
    max_median = np.max(median_array)
    norm_values = max_median / median_array
    return norm_values

def make_norm_image(image_stack, norm_values):
    """Create a normalization image stack based on the norm_values."""
    norm_image_stack = np.array([
        np.tile(norm_values, (image_stack.shape[2], 1)).T
        for _ in range(image_stack.shape[0])
    ], dtype=np.float32)
    return norm_image_stack

def main():
    # Select reference image
    Tk().withdraw()  # Hide Tkinter root window
    ref_path = filedialog.askopenfilename(title="Select a reference image", filetypes=[("TIFF files", "*.tif *.tiff")])
    if not ref_path:
        print("Reference image not selected.")
        return

    # Select sample stack
    sample_path = filedialog.askopenfilename(title="Select a sample image", filetypes=[("TIFF files", "*.tif *.tiff")])
    if not sample_path:
        print("Sample image not selected.")
        return

    print(f"Reference image: {ref_path}")
    print(f"Sample image: {sample_path}")

    # Load reference image and compute median array
    ref_image = np.array(Image.open(ref_path))
    print("Calculating medians...")
    median_array = get_median_array(ref_image)

    # Calculate normalization coefficients
    print("Calculating normalization values...")
    norm_values = get_normalizing_values(median_array)

    # Prepare normalization image based on sample stack dimensions
    print("Preparing normalization image...")
    sample_stack = tiff.imread(sample_path)
    norm_image_stack = make_norm_image(sample_stack, norm_values)

    # Apply normalization by multiplying each slice of the sample stack
    print("Applying normalization...")
    corrected_stack = sample_stack * norm_image_stack

    # Save the corrected image stack
    sample_path = Path(sample_path)
    save_path = Path(reset.animal_loc) / "ffc" / "l" / sample_path.name
    tiff.imwrite(save_path, corrected_stack.astype(np.uint8), photometric='minisblack')
    print(f"Normalized stack saved to {save_path}")

if __name__ == "__main__":
    main()


Reference image: /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/ffc/target.tif
Sample image: /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/ymerged/l/rbak006_l_2.tif
Calculating medians...
Calculating normalization values...
Preparing normalization image...
Applying normalization...
Normalized stack saved to /Volumes/BaffaloSSDPUTU3C1TB/rbak_data/rbak006/ffc/l/rbak006_l_2.tif
